In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from IPython.core.display import display, HTML
from IPython.lib.pretty import pprint
from itertools import chain, tee

In [3]:
from mockdown.model import View
from mockdown.display.view import display_view

In [4]:
from mockdown.model.view import ViewBuilder as V

sample_view = V("p", (0, 0, 100, 100), [
    V("a1", (10, 10, 45, 90), [
        V("b11", (20, 20, 35, 45)),
        V("b12", (20, 55, 35, 80))
    ]),
    V("a2", (55, 10, 90, 90), [
        V("b21", (65, 20, 80, 45)),
        V("b22", (65, 55, 80, 80))
    ])
]).to_view()

In [5]:
html_str = str(display_view(sample_view, scale=3))
display(HTML(html_str))

In [6]:
from mockdown.visibility import visible_pairs

edge_pairs = visible_pairs(sample_view, deep=True)
anchor_pairs = [(e1.anchor, e2.anchor) for (e1, e2) in edge_pairs]

In [7]:
html_str = str(display_view(sample_view, visible_pairs=edge_pairs, scale=3))
display(HTML(html_str))

In [8]:
from mockdown.logic import valid_constraints
constraints = list(valid_constraints(sample_view, anchor_pairs))

for constraint in constraints:
    print(str(constraint))

a2.left ≤ 1.0 * a1.right + 0 (priority=1000, samples=0, kind=spacing)
a2.left ≥ 1.0 * a1.right + 0 (priority=1000, samples=0, kind=spacing)
b12.top ≤ 1.0 * b11.bottom + 0 (priority=1000, samples=0, kind=spacing)
b12.top ≥ 1.0 * b11.bottom + 0 (priority=1000, samples=0, kind=spacing)
b22.top ≤ 1.0 * b21.bottom + 0 (priority=1000, samples=0, kind=spacing)
b22.top ≥ 1.0 * b21.bottom + 0 (priority=1000, samples=0, kind=spacing)
a1.top ≤ 1.0 * p.top + 0 (priority=1000, samples=0, kind=spacing)
a1.top ≥ 1.0 * p.top + 0 (priority=1000, samples=0, kind=spacing)
a1.left ≤ 1.0 * p.left + 0 (priority=1000, samples=0, kind=spacing)
a1.left ≥ 1.0 * p.left + 0 (priority=1000, samples=0, kind=spacing)
a1.bottom ≤ 1.0 * p.bottom + 0 (priority=1000, samples=0, kind=spacing)
a1.bottom ≥ 1.0 * p.bottom + 0 (priority=1000, samples=0, kind=spacing)
a2.top ≤ 1.0 * p.top + 0 (priority=1000, samples=0, kind=spacing)
a2.top ≥ 1.0 * p.top + 0 (priority=1000, samples=0, kind=spacing)
a2.bottom ≤ 1.0 * p.bottom +

In [9]:
html_str = str(display_view(sample_view, constraints=constraints, scale=5))
display(HTML(html_str))